In [17]:
from my_functions import*

In [18]:
file= uproot.open("/home/ruben/Escritorio/BachelorThesisRuben/Data/Muon_trees.root") #opening the Root file with Uproot 
MuonTree_ZeroBias=file["MuonTree_ZeroBias;1"]
MuonTree_Zmumu=file["MuonTree_Zmumu;1"]

In [19]:
res=energy_cut(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_eta"].array())

energy_cut: Only 5.79% of the events have survived the cut with a lower cut of 14000 and no upper cut


In [20]:
res

<Array [[-1.24], [-0.484], ..., [2.03], [2.45]] type='11575 * var * float32'>